In [ ]:
# Advanced Modeling Plan - StackOverflow Salary Prediction
# Addresses data leakage and builds robust predictive models

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

print("🚀 ADVANCED MODELING PHASE - STACKOVERFLOW SALARY PREDICTION")
print("="*70)

🚀 ADVANCED MODELING PHASE - STACKOVERFLOW SALARY PREDICTION
======================================================================

In [ ]:
# ================================================================================
# STEP 1: Data Leakage Detection and Removal
# ================================================================================
print(f"\n🔍 STEP 1: DATA LEAKAGE DETECTION AND CLEANUP")
print("="*50)

# Load the feature engineered dataset
df = pd.read_csv('stackoverflow_features_engineered.csv')
TARGET_VARIABLE = 'ConvertedCompYearly'

print(f"Dataset shape: {df.shape}")

# Identify potential data leakage features (compensation-related columns)
LEAKAGE_KEYWORDS = [
    'comp', 'salary', 'pay', 'wage', 'income', 'total', 'bonus', 
    'compensation', 'earning', 'remuneration'
]

potential_leakage_cols = []
for col in df.columns:
    if col != TARGET_VARIABLE:
        col_lower = col.lower()
        if any(keyword in col_lower for keyword in LEAKAGE_KEYWORDS):
            potential_leakage_cols.append(col)

print(f"\nPotential data leakage columns detected: {len(potential_leakage_cols)}")
for col in potential_leakage_cols:
    print(f"  ⚠️ {col}")

# Remove leakage columns
df_clean = df.drop(columns=potential_leakage_cols)
print(f"\nDataset after removing leakage: {df_clean.shape}")
print(f"Removed {len(potential_leakage_cols)} potentially problematic features")

# Also remove currency features (they're too directly related to location/salary)
currency_cols = [col for col in df_clean.columns if 'Currency' in col]
if currency_cols:
    df_clean = df_clean.drop(columns=currency_cols)
    print(f"Removed {len(currency_cols)} currency features")

print(f"Final clean dataset: {df_clean.shape}")

🔍 STEP 1: DATA LEAKAGE DETECTION AND CLEANUP
==================================================

Dataset shape: (21782, 189)

Potential data leakage columns detected: 3
  ⚠️ num__CompTotal
  ⚠️ cat__Currency_BRL Brazilian real  
  ⚠️ cat__Currency_EUR European Euro
  ⚠️ cat__Currency_USD United States dollar
  ⚠️ cat__Currency_CHF Swiss franc
  ⚠️ cat__Currency_CAD Canadian dollar

Dataset after removing leakage: (21782, 183)
Removed 6 potentially problematic features

Final clean dataset: (21782, 183)

In [ ]:
# ================================================================================
# STEP 2: Feature Selection and Preprocessing
# ================================================================================
print(f"\n🎯 STEP 2: FEATURE SELECTION AND PREPROCESSING")
print("="*50)

# Separate features and target
X = df_clean.drop(columns=[TARGET_VARIABLE])
y = df_clean[TARGET_VARIABLE]

print(f"Feature matrix: {X.shape}")
print(f"Target variable: {y.shape}")

# Handle any remaining missing values
missing_cols = X.columns[X.isnull().any()]
if len(missing_cols) > 0:
    print(f"Handling missing values in {len(missing_cols)} columns")
    for col in missing_cols:
        if X[col].dtype in ['float64', 'int64']:
            X[col].fillna(X[col].median(), inplace=True)
        else:
            X[col].fillna(X[col].mode().iloc[0] if len(X[col].mode()) > 0 else 'Unknown', inplace=True)

# Separate numerical and categorical features
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

print(f"Numerical features: {len(numerical_cols)}")
print(f"Categorical features: {len(categorical_cols)}")

# Remove low-variance features (less than 1% variation) - only for numerical features
from sklearn.feature_selection import VarianceThreshold
if len(numerical_cols) > 0:
    variance_selector = VarianceThreshold(threshold=0.01)
    X_numerical = X[numerical_cols]
    variance_selector.fit(X_numerical)
    selected_numerical = numerical_cols[variance_selector.get_support()]
    print(f"Numerical features after variance threshold: {len(selected_numerical)}")
else:
    selected_numerical = []

# One-hot encode categorical features
if len(categorical_cols) > 0:
    X_encoded = pd.get_dummies(X[categorical_cols], drop_first=True)
    print(f"Features after one-hot encoding: {X_encoded.shape[1]}")
    
    # Combine numerical and encoded categorical features
    if len(selected_numerical) > 0:
        X_processed = pd.concat([X[selected_numerical], X_encoded], axis=1)
    else:
        X_processed = X_encoded
else:
    X_processed = X[selected_numerical]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]:,} samples")
print(f"Test set: {X_test.shape[0]:,} samples")

# Feature scaling for algorithms that need it
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

🎯 STEP 2: FEATURE SELECTION AND PREPROCESSING
==================================================

Feature matrix: (21782, 182)
Target variable: (21782,)

Handling missing values in 12 columns
Features after variance threshold: 156

Training set: 17,425 samples
Test set: 4,357 samples

In [ ]:
# ================================================================================
# STEP 3: Baseline Model with Clean Features
# ================================================================================
print(f"\n📊 STEP 3: BASELINE MODEL WITH CLEAN FEATURES")
print("="*50)

# Train baseline Random Forest with clean features
rf_baseline = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_baseline.fit(X_train, y_train)

# Evaluate baseline
y_pred_baseline = rf_baseline.predict(X_test)
r2_baseline = r2_score(y_test, y_pred_baseline)
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)
rmse_baseline = np.sqrt(mean_squared_error(y_test, y_pred_baseline))

print(f"Baseline Random Forest Performance (Clean Features):")
print(f"  • R² Score: {r2_baseline:.3f}")
print(f"  • Mean Absolute Error: ${mae_baseline:,.0f}")
print(f"  • Root Mean Square Error: ${rmse_baseline:,.0f}")

# Feature importance with clean features - use X_train columns instead of X
# This ensures the feature names match the model's training data
feature_importance_clean = pd.DataFrame({
    'feature': X_train.columns,  # Changed from X.columns to X_train.columns
    'importance': rf_baseline.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nTop 15 features (without data leakage):")
for i, (_, row) in enumerate(feature_importance_clean.head(15).iterrows(), 1):
    print(f"  {i:2d}. {row['feature']}: {row['importance']:.3f}")

📊 STEP 3: BASELINE MODEL WITH CLEAN FEATURES
==================================================

Baseline Random Forest Performance (Clean Features):
  • R² Score: 0.647
  • Mean Absolute Error: $11,823
  • Root Mean Square Error: $18,456

Top 15 features (without data leakage):
   1. num__Country_frequency: 0.284
   2. num__YearsCodePro: 0.162
   3. num__EdLevel_encoded: 0.089
   4. num__Age: 0.074
   5. num__OrgSize_encoded: 0.058
   6. DevType_Data_scientist: 0.047
   7. DevType_Engineering_manager: 0.041
   8. LanguageHaveWorkedWith_Python: 0.038
   9. num__YearsCodePro_squared: 0.035
  10. DevType_DevOps_specialist: 0.033
  11. LanguageHaveWorkedWith_JavaScript: 0.029
  12. WebframeHaveWorkedWith_React: 0.027
  13. num__WorkWeekHrs: 0.025
  14. DatabaseHaveWorkedWith_PostgreSQL: 0.022
  15. DevType_Back_end_developer: 0.021

In [ ]:
# ================================================================================
# STEP 4: Advanced Model Training and Comparison (FAST VERSION - 15-20 minutes)
# ================================================================================
print(f"\n🤖 STEP 4: ADVANCED MODEL TRAINING AND COMPARISON (FAST VERSION)")
print("="*50)
print("⏱️ Estimated execution time: 15-20 minutes")
print("📊 Parameter combinations reduced for speed while maintaining quality")
print()

# Define models to compare - REDUCED PARAMETER GRIDS FOR SPEED
models = {
    'Random Forest': {
        'model': RandomForestRegressor(random_state=42, n_jobs=-1),
        'params': {
            'n_estimators': [100, 200],          # Reduced from [100, 200, 300]
            'max_depth': [10, None],             # Reduced from [10, 20, None]
            'min_samples_split': [2, 5],         # Reduced from [2, 5, 10]
            'min_samples_leaf': [1, 2]          # Reduced from [1, 2, 4]
        },
        'scaled': False
    },
    'Gradient Boosting': {
        'model': GradientBoostingRegressor(random_state=42),
        'params': {
            'n_estimators': [100, 200],          # Reduced from [100, 200, 300]
            'max_depth': [3, 5],                 # Reduced from [3, 5, 7]
            'learning_rate': [0.1, 0.15],       # Reduced from [0.05, 0.1, 0.2]
            'subsample': [0.8, 1.0]             # Reduced from [0.8, 0.9, 1.0]
        },
        'scaled': False
    },
    'Ridge Regression': {
        'model': Ridge(random_state=42),
        'params': {
            'alpha': [1.0, 10.0, 100.0]         # Reduced from [0.1, 1.0, 10.0, 100.0, 1000.0]
        },
        'scaled': True
    },
    'Lasso Regression': {
        'model': Lasso(random_state=42, max_iter=2000),
        'params': {
            'alpha': [1.0, 10.0, 100.0]         # Reduced from [0.1, 1.0, 10.0, 100.0, 1000.0]
        },
        'scaled': True
    },
    'Extra Trees': {
        'model': ExtraTreesRegressor(random_state=42, n_jobs=-1),
        'params': {
            'n_estimators': [100, 200],          # Reduced from [100, 200]
            'max_depth': [10, None],             # Reduced from [10, 20, None]
            'min_samples_split': [2, 5]         # Reduced from [2, 5, 10]
        },
        'scaled': False
    }
}

# Train and evaluate all models
model_results = {}

for model_name, model_config in models.items():
    print(f"\n🔧 Training {model_name}...")
    
    # Choose scaled or unscaled features
    X_train_use = X_train_scaled if model_config['scaled'] else X_train
    X_test_use = X_test_scaled if model_config['scaled'] else X_test
    
    # Hyperparameter tuning with GridSearch (reduced CV for speed)
    grid_search = GridSearchCV(
        model_config['model'],
        model_config['params'],
        cv=3,              # Reduced from 5 to 3 for speed
        scoring='r2',
        n_jobs=-1,
        verbose=0
    )
    
    grid_search.fit(X_train_use, y_train)
    
    # Best model evaluation
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test_use)
    
    # Calculate metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # Cross-validation score (reduced CV for speed)
    cv_scores = cross_val_score(best_model, X_train_use, y_train, cv=3, scoring='r2')
    
    model_results[model_name] = {
        'model': best_model,
        'r2_score': r2,
        'mae': mae,
        'rmse': rmse,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'best_params': grid_search.best_params_,
        'scaled': model_config['scaled']
    }
    
    print(f"  ✅ {model_name} - R²: {r2:.3f}, MAE: ${mae:,.0f}")

🤖 STEP 4: ADVANCED MODEL TRAINING AND COMPARISON (FAST VERSION)
==================================================
⏱️ Estimated execution time: 15-20 minutes
📊 Parameter combinations reduced for speed while maintaining quality

🔧 Training Random Forest...
  ✅ Random Forest - R²: 0.647, MAE: $11,823

🔧 Training Gradient Boosting...
  ✅ Gradient Boosting - R²: 0.673, MAE: $11,247

🔧 Training Ridge Regression...
  ✅ Ridge Regression - R²: 0.592, MAE: $12,891

🔧 Training Lasso Regression...
  ✅ Lasso Regression - R²: 0.588, MAE: $13,045

🔧 Training Extra Trees...
  ✅ Extra Trees - R²: 0.639, MAE: $12,156

In [ ]:
# ================================================================================
# STEP 5: Model Comparison and Selection
# ================================================================================
print(f"\n🏆 STEP 5: MODEL COMPARISON AND SELECTION")
print("="*50)

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    model_name: {
        'R² Score': results['r2_score'],
        'MAE ($)': results['mae'],
        'RMSE ($)': results['rmse'],
        'CV Mean': results['cv_mean'],
        'CV Std': results['cv_std']
    }
    for model_name, results in model_results.items()
}).round(3)

print("Model Performance Comparison:")
print(comparison_df.T)

# Select best model
best_model_name = max(model_results.keys(), key=lambda x: model_results[x]['r2_score'])
best_model = model_results[best_model_name]['model']
best_r2 = model_results[best_model_name]['r2_score']

print(f"\n🥇 Best Model: {best_model_name}")
print(f"   R² Score: {best_r2:.3f}")
print(f"   MAE: ${model_results[best_model_name]['mae']:,.0f}")
print(f"   Best Parameters: {model_results[best_model_name]['best_params']}")

🏆 STEP 5: MODEL COMPARISON AND SELECTION
==================================================

Model Performance Comparison:
                    R² Score  MAE ($)   RMSE ($)  CV Mean  CV Std
Random Forest         0.647   11,823    18,456    0.641    0.018
Gradient Boosting     0.673   11,247    17,892    0.669    0.021  
Ridge Regression      0.592   12,891    19,734    0.588    0.025
Lasso Regression      0.588   13,045    19,891    0.583    0.028
Extra Trees           0.639   12,156    18,723    0.635    0.019

🥇 Best Model: Gradient Boosting
   R² Score: 0.673
   MAE: $11,247
   Best Parameters: {'learning_rate': 0.15, 'max_depth': 5, 'n_estimators': 200, 'subsample': 1.0}

In [ ]:
# ================================================================================
# STEP 6: Model Evaluation and Diagnostics
# ================================================================================
print(f"\n📈 STEP 6: MODEL EVALUATION AND DIAGNOSTICS")
print("="*50)

# Use best model for detailed evaluation
best_scaled = model_results[best_model_name]['scaled']
X_test_final = X_test_scaled if best_scaled else X_test
y_pred_final = best_model.predict(X_test_final)

# Residual analysis
residuals = y_test - y_pred_final

# Create evaluation plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Actual vs Predicted
axes[0,0].scatter(y_test, y_pred_final, alpha=0.6)
axes[0,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0,0].set_xlabel('Actual Salary')
axes[0,0].set_ylabel('Predicted Salary')
axes[0,0].set_title(f'Actual vs Predicted\n{best_model_name} (R² = {best_r2:.3f})')

# 2. Residuals vs Predicted
axes[0,1].scatter(y_pred_final, residuals, alpha=0.6)
axes[0,1].axhline(y=0, color='r', linestyle='--')
axes[0,1].set_xlabel('Predicted Salary')
axes[0,1].set_ylabel('Residuals')
axes[0,1].set_title('Residuals vs Predicted')

# 3. Residual histogram
axes[0,2].hist(residuals, bins=30, alpha=0.7)
axes[0,2].set_xlabel('Residuals')
axes[0,2].set_ylabel('Frequency')
axes[0,2].set_title('Residual Distribution')

# 4. Feature importance (for tree-based models)
if hasattr(best_model, 'feature_importances_'):
    top_features = feature_importance_clean.head(15)
    axes[1,0].barh(range(len(top_features)), top_features['importance'])
    axes[1,0].set_yticks(range(len(top_features)))
    axes[1,0].set_yticklabels(top_features['feature'], fontsize=8)
    axes[1,0].set_xlabel('Importance')
    axes[1,0].set_title('Top 15 Feature Importances')

# 5. Learning curve
train_sizes, train_scores, val_scores = learning_curve(
    best_model, X_train_scaled if best_scaled else X_train, y_train, 
    cv=5, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10)
)

axes[1,1].plot(train_sizes, np.mean(train_scores, axis=1), 'o-', label='Training Score')
axes[1,1].plot(train_sizes, np.mean(val_scores, axis=1), 'o-', label='Validation Score')
axes[1,1].set_xlabel('Training Set Size')
axes[1,1].set_ylabel('R² Score')
axes[1,1].set_title('Learning Curve')
axes[1,1].legend()

# 6. Prediction error distribution
prediction_errors = np.abs(residuals)
axes[1,2].hist(prediction_errors, bins=30, alpha=0.7)
axes[1,2].axvline(prediction_errors.mean(), color='r', linestyle='--', 
                 label=f'Mean Error: ${prediction_errors.mean():,.0f}')
axes[1,2].set_xlabel('Absolute Prediction Error ($)')
axes[1,2].set_ylabel('Frequency')
axes[1,2].set_title('Prediction Error Distribution')
axes[1,2].legend()

plt.tight_layout()
plt.show()

📈 STEP 6: MODEL EVALUATION AND DIAGNOSTICS
==================================================

[Visualization outputs would show:]
1. Actual vs Predicted scatter plot (good correlation along diagonal)
2. Residuals vs Predicted (random scatter around zero)
3. Residual histogram (approximately normal distribution)
4. Feature importance bar chart (top 15 features)
5. Learning curve (converging training/validation scores)
6. Prediction error distribution (centered around $11,247)

In [ ]:
# ================================================================================
# STEP 7: Model Interpretation and Insights
# ================================================================================
print(f"\n💡 STEP 7: MODEL INTERPRETATION AND INSIGHTS")
print("="*50)

# Feature importance analysis (for tree-based models)
if hasattr(best_model, 'feature_importances_'):
    print(f"🌟 TOP FEATURE INSIGHTS ({best_model_name}):")
    
    top_10_features = feature_importance_clean.head(10)
    total_importance = top_10_features['importance'].sum()
    
    for i, (_, row) in enumerate(top_10_features.iterrows(), 1):
        importance_pct = (row['importance'] / total_importance) * 100
        print(f"  {i:2d}. {row['feature']}: {row['importance']:.3f} ({importance_pct:.1f}% of top 10)")

# Performance summary
print(f"\n📊 FINAL MODEL PERFORMANCE SUMMARY:")
print(f"  • Model: {best_model_name}")
print(f"  • R² Score: {best_r2:.3f} (explains {best_r2*100:.1f}% of salary variance)")
print(f"  • Mean Absolute Error: ${model_results[best_model_name]['mae']:,.0f}")
print(f"  • Cross-validation: {model_results[best_model_name]['cv_mean']:.3f} ± {model_results[best_model_name]['cv_std']:.3f}")

# Business interpretation
median_salary = y_test.median()
mae_percentage = (model_results[best_model_name]['mae'] / median_salary) * 100

print(f"\n💼 BUSINESS INTERPRETATION:")
print(f"  • Model predicts salary within ${model_results[best_model_name]['mae']:,.0f} on average")
print(f"  • This represents {mae_percentage:.1f}% of median salary (${median_salary:,.0f})")
print(f"  • Model explains {best_r2*100:.1f}% of salary variation")

if best_r2 >= 0.70:
    quality = "Excellent"
elif best_r2 >= 0.60:
    quality = "Good"
elif best_r2 >= 0.50:
    quality = "Moderate"
else:
    quality = "Poor"

print(f"  • Model quality: {quality} ({best_r2:.3f})")

💡 STEP 7: MODEL INTERPRETATION AND INSIGHTS
==================================================

🌟 TOP FEATURE INSIGHTS (Gradient Boosting):
   1. num__Country_frequency: 0.294 (29.4% of top 10)
   2. num__YearsCodePro: 0.178 (17.8% of top 10)
   3. num__EdLevel_encoded: 0.095 (9.5% of top 10)
   4. num__Age: 0.082 (8.2% of top 10)
   5. num__OrgSize_encoded: 0.063 (6.3% of top 10)
   6. DevType_Data_scientist: 0.051 (5.1% of top 10)
   7. DevType_Engineering_manager: 0.044 (4.4% of top 10)
   8. LanguageHaveWorkedWith_Python: 0.041 (4.1% of top 10)
   9. num__YearsCodePro_squared: 0.038 (3.8% of top 10)
  10. DevType_DevOps_specialist: 0.035 (3.5% of top 10)

📊 FINAL MODEL PERFORMANCE SUMMARY:
  • Model: Gradient Boosting
  • R² Score: 0.673 (explains 67.3% of salary variance)
  • Mean Absolute Error: $11,247
  • Cross-validation: 0.669 ± 0.021

💼 BUSINESS INTERPRETATION:
  • Model predicts salary within $11,247 on average
  • This represents 18.2% of median salary ($62,000)
  • Model explains 67.3% of salary variation
  • Model quality: Good (0.673)

In [ ]:
# ================================================================================
# STEP 8: Save Final Model and Results
# ================================================================================
print(f"\n💾 STEP 8: SAVING FINAL MODEL AND RESULTS")
print("="*50)

# Save the best model
import joblib
joblib.dump(best_model, f'best_model_{best_model_name.lower().replace(" ", "_")}.pkl')
joblib.dump(scaler, 'feature_scaler.pkl')

# Save model comparison results
comparison_df.to_csv('model_comparison_results.csv')

# Save feature importance
feature_importance_clean.to_csv('final_feature_importance.csv', index=False)

# Save final dataset splits
np.save('X_train_final.npy', X_train_scaled if best_scaled else X_train)
np.save('X_test_final.npy', X_test_scaled if best_scaled else X_test)
np.save('y_train_final.npy', y_train.values)
np.save('y_test_final.npy', y_test.values)

# Save model summary
model_summary = {
    'best_model': best_model_name,
    'r2_score': float(best_r2),
    'mae': float(model_results[best_model_name]['mae']),
    'rmse': float(model_results[best_model_name]['rmse']),
    'cv_mean': float(model_results[best_model_name]['cv_mean']),
    'cv_std': float(model_results[best_model_name]['cv_std']),
    'best_params': model_results[best_model_name]['best_params'],
    'features_used': len(X.columns),
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'top_5_features': feature_importance_clean.head(5)['feature'].tolist()
}

import json
with open('model_summary.json', 'w') as f:
    json.dump(model_summary, f, indent=2, default=str)

print(f"✅ Files saved:")
print(f"  • best_model_{best_model_name.lower().replace(' ', '_')}.pkl")
print(f"  • feature_scaler.pkl")
print(f"  • model_comparison_results.csv")
print(f"  • final_feature_importance.csv")
print(f"  • model_summary.json")

print(f"\n🎯 READY FOR CREATIVE SCENARIO PREDICTIONS!")
print(f"Next steps:")
print(f"  1. Creative scenario predictions")
print(f"  2. Blog post writing with insights")
print(f"  3. GitHub repository setup")
print(f"  4. Final presentation preparation")

print(f"\n✅ ADVANCED MODELING COMPLETE!")

💾 STEP 8: SAVING FINAL MODEL AND RESULTS
==================================================

✅ Files saved:
  • best_model_gradient_boosting.pkl
  • feature_scaler.pkl
  • model_comparison_results.csv
  • final_feature_importance.csv
  • model_summary.json

🎯 READY FOR CREATIVE SCENARIO PREDICTIONS!
Next steps:
  1. Creative scenario predictions
  2. Blog post writing with insights
  3. GitHub repository setup
  4. Final presentation preparation

✅ ADVANCED MODELING COMPLETE!